In [1]:
import tensorflow as tf

In [2]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)
print node1, node2


Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [3]:
sess = tf.Session()
print sess.run([node1, node2])

[3.0, 4.0]


In [4]:
node3 = tf.add(node1, node2)
print node3
print sess.run(node3)
file_writer = tf.summary.FileWriter('logs', sess.graph)

Tensor("Add:0", shape=(), dtype=float32)
7.0


In [5]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b


In [6]:
print sess.run(adder_node, {a: [3], b: [123.91232, 12]})
tf.summary.FileWriter('logs', sess.graph)

[ 126.912323   15.      ]


In [7]:
add_and_triple = adder_node * 3
print sess.run(add_and_triple, {a: 4, b: 4.5})

25.5


In [8]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

In [9]:
init = tf.global_variables_initializer()
sess.run(init)

In [10]:
print sess.run(linear_model, {x: [1,2,3,4]})

[ 0.          0.30000001  0.60000002  0.90000004]


In [11]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print sess.run(loss, {x: [1,2,3,4], y: [0, -1, -2, -3]})

23.66


In [12]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print sess.run(loss, {x: [1,2,3,4], y:[0, -1, -2, -3]})

0.0


In [13]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [14]:
sess.run(init)
for i in range(1000):
    sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})
print sess.run([W, b])

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [15]:
import numpy as np
import tensorflow as tf

# model param
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)

# model input and output
x = tf.placeholder(tf.float32)
linear_model = (W * x) + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y))
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss=loss)

x_train = [1,2,3,4]
y_train = [0, -1, -2, -3]
#training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(1000):
    sess.run(train, {x: x_train, y: y_train})
    
# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print curr_W, curr_b, curr_loss
tf.summary.FileWriter('logs', sess.graph)

[-0.9999969] [ 0.99999082] 5.69997e-11


In [16]:
import tensorflow as tf
import numpy as np

features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.contrib.learn.io.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000
)
eval_input_fn = tf.contrib.learn.io.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000 
)
estimator.fit(input_fn=input_fn, steps=1000)
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print train_loss
print eval_loss

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9fcbee3350>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:S

In [17]:
import numpy as np
import tensorflow as tf

def model(features, labels, mode):
    W = tf.get_variable('W', [1], dtype=tf.float64)
    b = tf.get_variable('b', [1], dtype=tf.float64)
    y = W * features['x'] + b
    
    loss = tf.reduce_sum(tf.square(y - labels))
    
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss=loss),
                    tf.assign_add(global_step, 1))
    
    return tf.contrib.learn.ModelFnOps(
        mode=mode, predictions=y, loss=loss, train_op=train
    )
estimator = tf.contrib.learn.Estimator(model_fn=model)

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.contrib.learn.io.numpy_input_fn(
    {'x': x_train}, y_train, batch_size=4, num_epochs=1000
)
estimator.fit(input_fn=input_fn, steps=1000)

train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print train_loss, eval_loss

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9fcb4c2810>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp45Q8Ra/model.ckpt.
INFO:tensorflow:loss = 7.35849958964, step = 1
INFO:tensorflow:global_step/sec: 1153.32
INFO:tensorflow:loss = 0.332543907749, step = 101 (0.088 sec)
INFO:tensorflow:global_step/sec: 1290.84
INFO:tensorflow:loss = 0.0308206380442, step = 201 (0.079 sec)
INFO:tensorflow:glo